# TNDC algorithm

In [ ]:
import opyplus as op
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import os

# Definition
    # cvrsme
    # mbe
    # cvrsme_night 
    # cvrsme_day
    # Infil: Infiltration rate 
    # SHGC: solar heat gain coefficient
# Load your building model
model = op.load_model('your_model.idf')

# Define your parameters
Infil_values = np.linspace(Infil_min, Infil_max, num_steps)  # replace with your range and steps
SHGC_values = np.linspace(SHGC_min, SHGC_max, num_steps)  # replace with your range and steps

# Initialize variables to store best parameters and minimum CVRSME
best_Infil = None
best_SHGC = None

# replace 'inf' with minimum CVRSME and MBE requirement
min_cvrsme = float('inf') 
min_mbe = float('inf')  
min_cvrsme_day = float('inf') 
min_mbe_day = float('inf') 
min_cvrsme_night = float('inf') 
min_mbe_night = float('inf')  

# Initialize DataFrame to store CVRSME values
df = pd.DataFrame(columns=['Iteration', 'Infil', 'SHGC', 'CVRSME'])

# First loop: calibrate Infil with initial SHGC
for Infil in Infil_values:
    # Set parameter Infil in your model
    model.set_parameter('Infil', Infil)
    
    # Run the model for night
    model.run_night()
    
    # Calculate CVRSME and MBE for night
    cvrsme_night = model.calculate_cvrsme()
    mbe_night = model.calculate_mbe()

    
    # Update best_Infil and min_cvrsme_night if necessary
    if cvrsme_night < min_cvrsme_night and mbe_night < min_mbe_night:
        best_Infil = Infil
        min_cvrsme_night = cvrsme_night

# Second loop: calibrate SHGC with best Infil
model.set_parameter('Infil', best_Infil)
for SHGC in SHGC_values:
    # Set parameter SHGC in your model
    model.set_parameter('SHGC', SHGC)
    
    # Run the model for day
    model.run_day()
    
    # Calculate CVRSME and MBE for day
    cvrsme_day = model.calculate_cvrsme()
    mbe_day = model.calculate_mbe()

    
    # Run the model for total
    model.run_total()
    
    # Calculate total CVRSME and MBE 
    cvrsme = model.calculate_cvrsme()
    mbe = model.calculate_mbe()
    
    
    # Update best_SHGC and min_cvrsme if necessary
    if cvrsme_day < min_cvrsme_day and mbe_day < min_mbe_day: 
        if cvrsme < min_cvrsme and mbe < min_mbe:
            best_SHGC = SHGC
            min_cvrsme = cvrsme

# Third loop: check Infil and SHGC for total day
model.set_parameter('Infil', best_Infil)
model.set_parameter('SHGC', best_SHGC)
model.run_total()
cvrsme = model.calculate_cvrsme()
mbe = model.calculate_mbe()


# Print the best parameters and minimum CVRSME
print(f'Best Infil: {best_Infil}, Best SHGC: {best_SHGC}, CVRSME: {cvrsme}, MBE: {mbe}')